# Section 3: Python & Hugging Face

## Load the Dataset

In [ ]:
import pandas as pd

df = pd.read_csv('../data/traininig-dataset.csv')

print(f"Dataset loaded: {len(df)} samples")

print(f"\nLabel distribution:")
label_counts = df['label'].value_counts().reset_index()
label_counts.columns = ['label', 'count']
print(label_counts.to_string(index=False))

print(f"\nFirst few rows:")
print(df.head(3).to_string(index=False))

Dataset loaded: 50 samples

Label distribution:
 label  count
     1     26
     0     24

First few rows:
  note_id                                               text  label
0   N2013  Patient reports flooding during menses since m...      1
1   N2015  Patient reports heavy menses since stopping OC...      1
2   N2022  Patient reports HMB for the past 6 months, des...      1


## Tokenize the Text Field

## Train a Small Classifier